# Agregando datos con pandas y numpy

## Sobre los datos
En este cuaderno trabajaremos con 2 conjuntos de datos:
- La cotización de las acciones de Facebook a lo largo de 2018 (obtenida mediante la [`stock_analysis` package](https://github.com/stefmolin/stock-analysis)).
- Datos meteorológicos diarios de Nueva York [National Centers for Environmental Information (NCEI) API](https://www.ncdc.noaa.gov/cdo-web/webservices/v2).

*Nota: El NCEI forma parte de la Administración Nacional Oceánica y Atmosférica (NOAA) y, como puede ver en la URL de la API, este recurso se creó cuando el NCEI se llamaba NCDC. Si la URL de este recurso cambiara en el futuro, puede buscar "NCEI weather API" para encontrar la actualizada.*

## Antecedentes de los datos meteorológicos

Significado de los datos:
- `AWND`: velocidad media del viento
- `PRCP`: precipitación en milímetros
- `SNOW`: nevadas en milímetros
- SNWD`: profundidad de la nieve en milímetros
- TMAX`: temperatura máxima diaria en grados Celsius
- TMIN`: temperatura mínima diaria en grados Celsius

## Setup

In [1]:
import numpy as np
import pandas as pd

fb = pd.read_csv('data/fb_2018.csv', index_col='date', parse_dates=True).assign(
    trading_volume=lambda x: pd.cut(x.volume, bins=3, labels=['low', 'med', 'high'])
)
fb.head()

,open,high,low,close,volume,trading_volume
date,,,,,,
2018-01-02,177.68,181.58,177.5500,181.42,18151903,low
2018-01-03,181.88,184.78,181.3300,184.67,16886563,low
2018-01-04,184.90,186.21,184.0996,184.33,13880896,low
2018-01-05,185.59,186.90,184.9300,186.85,13574535,low
2018-01-08,187.20,188.90,186.3300,188.28,17994726,low


In [2]:
weather = pd.read_csv('data/weather_by_station.csv', index_col='date', parse_dates=True)
weather.head()

,datatype,station,value,station_name
date,,,,
2018-01-01,PRCP,GHCND:US1CTFR0039,0.0,"STAMFORD 4.2 S, CT US"
2018-01-01,PRCP,GHCND:US1NJBG0015,0.0,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0015,0.0,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,PRCP,GHCND:US1NJBG0017,0.0,"GLEN ROCK 0.7 SSE, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0017,0.0,"GLEN ROCK 0.7 SSE, NJ US"


Antes de sumergirnos en cualquier cálculo, vamos a asegurarnos de que `pandas` no pondrá las cosas en notación científica. Modificaremos el formato de visualización de los flotantes. El formato que aplicaremos es `.2f`, que proporcionará al flotante 2 dígitos después del punto decimal:

In [3]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## Resumiendo DataFrames
Aprendimos a utilizar `agg()` en la sección [`2-dataframe_operations.ipynb`](./2-dataframe_operations.ipynb) cuando aprendimos acerca de los cálculos de ventanas; sin embargo, podemos llamar a esto en el marco de datos directamente para agregar su contenido en una sola serie:

In [4]:
fb.agg({
    'open': np.mean, 
    'high': np.max, 
    'low': np.min, 
    'close': np.mean, 
    'volume': np.sum
})

C:\Users\feder\AppData\Local\Temp\ipykernel_127356\2168324887.py:1: FutureWarning: The provided callable <function mean at 0x000001874A8080E0> is currently using Series.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  fb.agg({
C:\Users\feder\AppData\Local\Temp\ipykernel_127356\2168324887.py:1: FutureWarning: The provided callable <function max at 0x000001874A7FB6A0> is currently using Series.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  fb.agg({
C:\Users\feder\AppData\Local\Temp\ipykernel_127356\2168324887.py:1: FutureWarning: The provided callable <function min at 0x000001874A7FB7E0> is currently using Series.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  fb.agg({
C:\Users\feder\AppData\Local\Temp\ipykernel_12735

open            171.45
high            218.62
low             123.02
close           171.51
volume   6949682394.00
dtype: float64

Podemos utilizarlo para hallar el total de nevadas y precipitaciones registradas en Central Park en 2018:

In [5]:
weather.query('station == "GHCND:USW00094728"')\
    .pivot(columns='datatype', values='value')[['SNOW', 'PRCP']]\
    .sum()

datatype
SNOW   1007.00
PRCP   1665.30
dtype: float64

Esto equivale a pasar `'sum'` a `agg()`:

In [6]:
weather.query('station == "GHCND:USW00094728"')\
    .pivot(columns='datatype', values='value')[['SNOW', 'PRCP']]\
    .agg('sum')

datatype
SNOW   1007.00
PRCP   1665.30
dtype: float64

Tenga en cuenta que no estamos limitados a proporcionar una única agregación por columna. Podemos pasar una lista y obtendremos un marco de datos en lugar de una serie. Los valores nulos se colocan donde no tenemos un resultado de cálculo para mostrar:

In [7]:
fb.agg({
    'open': 'mean',
    'high': ['min', 'max'],
    'low': ['min', 'max'],
    'close': 'mean'
})

,open,high,low,close
mean,171.45,NaN,NaN,171.51
min,NaN,129.74,123.02,NaN
max,NaN,218.62,214.27,NaN


## Usando `groupby()`
A menudo no queremos agregar todo el marco de datos, sino grupos dentro de él. Para ello, podemos ejecutar `groupby()` antes de la agregación. Si agrupamos por la columna `trading_volume`, obtendremos una fila por cada uno de los valores que tome:

In [8]:
fb.groupby('trading_volume').mean()

C:\Users\feder\AppData\Local\Temp\ipykernel_127356\46027550.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  fb.groupby('trading_volume').mean()


,open,high,low,close,volume
trading_volume,,,,,
low,171.36,173.46,169.31,171.43,24547207.71
med,175.82,179.42,172.11,175.14,79072559.12
high,167.73,170.48,161.57,168.16,141924023.33


Después de llamar a `groupby()`, todavía podemos seleccionar columnas para la agregación:

In [9]:
fb.groupby('trading_volume')['close'].agg(['min', 'max', 'mean'])

C:\Users\feder\AppData\Local\Temp\ipykernel_127356\3607524933.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  fb.groupby('trading_volume')['close'].agg(['min', 'max', 'mean'])


,min,max,mean
trading_volume,,,
low,124.06,214.67,171.43
med,152.22,217.50,175.14
high,160.06,176.26,168.16


Todavía podemos proporcionar un diccionario especificando las agregaciones a realizar, pero pasando una lista para una columna resultará en un índice jerárquico para las columnas:

In [10]:
fb_agg = fb.groupby('trading_volume').agg({
    'open': 'mean',
    'high': ['min', 'max'],
    'low': ['min', 'max'],
    'close': 'mean'
})
fb_agg

C:\Users\feder\AppData\Local\Temp\ipykernel_127356\2667628102.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  fb_agg = fb.groupby('trading_volume').agg({


open   high           low         close
                 mean    min    max    min    max   mean
trading_volume                                          
low            171.36 129.74 216.20 123.02 212.60 171.43
med            175.82 162.85 218.62 150.75 214.27 175.14
high           167.73 161.10 180.13 149.02 173.75 168.16

El índice jerárquico en las columnas tiene el siguiente aspecto:

In [11]:
fb_agg.columns

MultiIndex([( 'open', 'mean'),
            ( 'high',  'min'),
            ( 'high',  'max'),
            (  'low',  'min'),
            (  'low',  'max'),
            ('close', 'mean')],
           )

Utilizando una comprensión de lista, podemos unir los niveles (en una tupla) con un `_` en cada iteración:

In [12]:
fb_agg.columns = ['_'.join(col_agg) for col_agg in fb_agg.columns]
fb_agg.head()

,open_mean,high_min,high_max,low_min,low_max,close_mean
trading_volume,,,,,,
low,171.36,129.74,216.20,123.02,212.60,171.43
med,175.82,162.85,218.62,150.75,214.27,175.14
high,167.73,161.10,180.13,149.02,173.75,168.16


Podemos agrupar en valores en el índice si le decimos a `groupby()`, que `nivel` usar:

In [13]:
weather.loc['2018-10'].query('datatype == "PRCP"')\
    .groupby(level=0).mean().head().squeeze()

TypeError: agg function failed [how->mean,dtype->object]

También podemos crear un objeto `Grouper`, que también puede enrollar las fechas en el índice. Aquí, encontramos la precipitación total trimestral por estación:

In [14]:
weather.query('datatype == "PRCP"').groupby(
    ['station_name', pd.Grouper(freq='Q')]
).sum().unstack().sample(5, random_state=1)

datatype  \
date                                                                2018-03-31   
station_name                                                                     
WANTAGH 1.1 NNE, NY US       PRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPR...   
STATEN ISLAND 1.4 SE, NY US  PRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPR...   
SYOSSET 2.0 SSW, NY US       PRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPR...   
STAMFORD 4.2 S, CT US        PRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPR...   
WAYNE TWP 0.8 SSW, NJ US      PRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCP   

                                                                                \
date                                                                2018-06-30   
station_name                                                                     
WANTAGH 1.1 NNE, NY US       PRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPR...   
STATEN ISLAND 1.4 SE, NY US  PRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPR...   
SYOSSET 2.0 SSW, NY US       PRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPR...   
STAMFORD 4.2 S, CT US        PRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPR...   
WAYNE TWP 0.8 SSW, NJ US     PRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPR...   

                                                                                \
date                                                                2018-09-30   
station_name                                                                     
WANTAGH 1.1 NNE, NY US       PRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPR...   
STATEN ISLAND 1.4 SE, NY US  PRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPR...   
SYOSSET 2.0 SSW, NY US       PRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPR...   
STAMFORD 4.2 S, CT US        PRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPR...   
WAYNE TWP 0.8 SSW, NJ US     PRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPR...   

                                                                                \
date                                                                2018-12-31   
station_name                                                                     
WANTAGH 1.1 NNE, NY US       PRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPR...   
STATEN ISLAND 1.4 SE, NY US  PRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPR...   
SYOSSET 2.0 SSW, NY US       PRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPR...   
STAMFORD 4.2 S, CT US        PRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPR...   
WAYNE TWP 0.8 SSW, NJ US     PRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPRCPPR...   

                                                                       station  \
date                                                                2018-03-31   
station_name                                                                     
WANTAGH 1.1 NNE, NY US       GHCND:US1NYNS0037GHCND:US1NYNS0037GHCND:US1NYN...   
STATEN ISLAND 1.4 SE, NY US  GHCND:US1NYRC0002GHCND:US1NYRC0002GHCND:US1NYR...   
SYOSSET 2.0 SSW, NY US       GHCND:US1NYNS0036GHCND:US1NYNS0036GHCND:US1NYN...   
STAMFORD 4.2 S, CT US        GHCND:US1CTFR0039GHCND:US1CTFR0039GHCND:US1CTF...   
WAYNE TWP 0.8 SSW, NJ US     GHCND:US1NJPS0015GHCND:US1NJPS0015GHCND:US1NJP...   

                                                                                \
date                                                                2018-06-30   
station_name                                                                     
WANTAGH 1.1 NNE, NY US       GHCND:US1NYNS0037GHCND:US1NYNS0037GHCND:US1NYN...   
STATEN ISLAND 1.4 SE, NY US  GHCND:US1NYRC0002GHCND:US1NYRC0002GHCND:US1NYR...   
SYOSSET 2.0 SSW, NY US       GHCND:US1NYNS0036GHCND:US1NYNS0036GHCND:US1NYN...   
STAMFORD 4.2 S, CT US        GHCND:US1CTFR0039GHCND:US1CTFR0039GHCND:US1CTF...   
WAYNE TWP 0.8 SSW, NJ US     GHCND:US1NJPS0015GHCND:US1NJPS0015GHCND:US1NJP...   

                                                                                \
date                                          

Tenga en cuenta que podemos utilizar `filter()` para excluir algunos grupos de la agregación. En este caso, sólo mantenemos los grupos cuyos nombres terminan en "NY US" en el atributo `name` del grupo, que en este caso es el nombre de la estación:

In [19]:
weather.groupby('station_name').filter( # station names with "NY US" in them
    lambda x: x.name.endswith('NY US')
).query('datatype == "SNOW"').groupby('station_name').sum().squeeze() # agregar y hacer una serie (squeeze)

,datatype,station,value
station_name,,,
"ALBERTSON 0.2 SSE, NY US",SNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSN...,GHCND:US1NYNS0042GHCND:US1NYNS0042GHCND:US1NYN...,1087.00
"AMITYVILLE 0.1 WSW, NY US",SNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSN...,GHCND:US1NYSF0089GHCND:US1NYSF0089GHCND:US1NYS...,434.00
"AMITYVILLE 0.6 NNE, NY US",SNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSN...,GHCND:US1NYSF0092GHCND:US1NYSF0092GHCND:US1NYS...,1072.00
"ARMONK 0.3 SE, NY US",SNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSN...,GHCND:US1NYWC0018GHCND:US1NYWC0018GHCND:US1NYW...,1504.00
"BROOKLYN 3.1 NW, NY US",SNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSN...,GHCND:US1NYKN0025GHCND:US1NYKN0025GHCND:US1NYK...,305.00
"CENTERPORT 0.9 SW, NY US",SNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSN...,GHCND:US1NYSF0061GHCND:US1NYSF0061GHCND:US1NYS...,799.00
"CENTERPORT, NY US",SNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSN...,GHCND:USC00301309GHCND:USC00301309GHCND:USC003...,1333.00
"ELMSFORD 0.8 SSW, NY US",SNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOW,GHCND:US1NYWC0019GHCND:US1NYWC0019GHCND:US1NYW...,863.00
"FLORAL PARK 0.4 W, NY US",SNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSNOWSN...,GHCND:US1NYNS0007GHCND:US1NYNS0007GHCND:US1NYN...,1015.00


Veamos cuáles son los meses con más precipitaciones. En primer lugar, tenemos que agrupar por días y calcular la media de las precipitaciones en todas las estaciones. Luego podemos agrupar por meses y sumar la precipitación resultante. Usamos `nlargest()` para obtener los 5 meses con mayor precipitación:

In [21]:
weather.query('datatype == "PRCP"')\
    .groupby(level=0).mean()\
    .groupby(pd.Grouper(freq='M')).sum().value.nlargest()

TypeError: agg function failed [how->mean,dtype->object]

Quizás el resultado anterior fue sorprendente. El refrán dice que "las lluvias de abril traen las flores de mayo"; sin embargo, abril no estaba entre los 5 primeros (ni tampoco mayo). La nieve contará para las precipitaciones, pero eso no explica por qué los meses de verano están por encima de abril. Busquemos los días que representaron un gran porcentaje de la precipitación en un mes determinado.

Para ello, tenemos que calcular la precipitación media diaria en todas las estaciones y, a continuación, hallar el total por mes. Este será el denominador. Sin embargo, para dividir los valores diarios por el total de su mes, necesitaremos una serie de dimensiones iguales. Esto significa que tendremos que utilizar `transform()`:

In [22]:
weather.query('datatype == "PRCP"')\
    .rename(dict(value='prcp'), axis=1)\
    .groupby(level=0).mean()\
    .groupby(pd.Grouper(freq='M'))\
    .transform(np.sum)['2018-01-28':'2018-02-03']

TypeError: agg function failed [how->mean,dtype->object]

Observe que tenemos el mismo valor repetido para cada día del mes al que pertenece. Esto nos permitirá calcular el porcentaje de la precipitación mensual que se produjo cada día y, a continuación, extraer los valores más grandes:

In [23]:
weather\
    .query('datatype == "PRCP"')\
    .rename(dict(value='prcp'), axis=1)\
    .groupby(level=0).mean()\
    .assign(
        total_prcp_in_month=lambda x: \
            x.groupby(pd.Grouper(freq='M')).transform(np.sum),
        pct_monthly_prcp=lambda x: \
            x.prcp.div(x.total_prcp_in_month)
    )\
    .nlargest(5, 'pct_monthly_prcp')

TypeError: agg function failed [how->mean,dtype->object]

`transform()` también se puede utilizar en dataframes. Podemos utilizarlo para normalizar fácilmente los datos:

In [24]:
fb[['open', 'high', 'low', 'close']]\
    .transform(lambda x: (x - x.mean()).div(x.std()))\
    .head()

,open,high,low,close
date,,,,
2018-01-02,0.32,0.41,0.41,0.50
2018-01-03,0.53,0.57,0.60,0.66
2018-01-04,0.68,0.65,0.74,0.64
2018-01-05,0.72,0.68,0.78,0.77
2018-01-08,0.80,0.79,0.85,0.84


## Tablas dinámicas y tablas cruzadas
Vimos pivotes en [`ch_03/4-reshaping_data.ipynb`](../ch_03/4-reshaping_data.ipynb); sin embargo, no pudimos proporcionar ninguna agregación. Con `pivot_table()`, obtenemos la media por defecto. En su forma más simple, proporcionamos una columna para colocar a lo largo de las columnas:

In [25]:
fb.pivot_table(columns='trading_volume')

trading_volume,low,med,high
close,171.43,175.14,168.16
high,173.46,179.42,170.48
low,169.31,172.11,161.57
open,171.36,175.82,167.73
volume,24547207.71,79072559.12,141924023.33


Al colocar el volumen de negociación en el índice, obtenemos la transposición:

In [26]:
fb.pivot_table(index='trading_volume')

,close,high,low,open,volume
trading_volume,,,,,
low,171.43,173.46,169.31,171.36,24547207.71
med,175.14,179.42,172.11,175.82,79072559.12
high,168.16,170.48,161.57,167.73,141924023.33


Con `pivot()`, tampoco hemos podido manejar índices multinivel o índices con valores repetidos. Por esta razón no hemos podido poner los datos meteorológicos en el formato ancho. El método `pivot_table()` resuelve este problema:

In [27]:
weather.reset_index().pivot_table(
    index=['date', 'station', 'station_name'], 
    columns='datatype', 
    values='value',
    aggfunc='median'
).reset_index().tail()

datatype,date,station,station_name,AWND,DAPR,MDPR,PGTM,PRCP,SNOW,SNWD,...,WSF5,WT01,WT02,WT03,WT04,WT05,WT06,WT08,WT09,WT11
28740,2018-12-31,GHCND:USW00054787,"FARMINGDALE REPUBLIC AIRPORT, NY US",5.00,NaN,NaN,2052.00,28.70,NaN,NaN,...,15.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28741,2018-12-31,GHCND:USW00094728,"NY CITY CENTRAL PARK, NY US",NaN,NaN,NaN,NaN,25.90,0.00,0.00,...,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28742,2018-12-31,GHCND:USW00094741,"TETERBORO AIRPORT, NJ US",1.70,NaN,NaN,1954.00,29.20,NaN,NaN,...,8.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28743,2018-12-31,GHCND:USW00094745,"WESTCHESTER CO AIRPORT, NY US",2.70,NaN,NaN,2212.00,24.40,NaN,NaN,...,11.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28744,2018-12-31,GHCND:USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",4.10,NaN,NaN,NaN,31.20,0.00,0.00,...,12.50,1.00,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Podemos utilizar la función `pd.crosstab()` para crear una tabla de frecuencias. Por ejemplo, si queremos ver cuántos días de negociación de bajo, medio y alto volumen han tenido las acciones de Facebook cada mes, podemos utilizar crosstab:

In [28]:
pd.crosstab(
    index=fb.trading_volume,
    columns=fb.index.month,
    colnames=['month'] # name the columns index
)

month,1,2,3,4,5,6,7,8,9,10,11,12
trading_volume,,,,,,,,,,,,
low,20,19,15,20,22,21,18,23,19,23,21,19
med,1,0,4,1,0,0,2,0,0,0,0,0
high,0,0,2,0,0,0,1,0,0,0,0,0


Podemos normalizar con los totales de fila o columna con el parámetro `normalize`. Esto muestra el porcentaje del total:

In [29]:
pd.crosstab(
    index=fb.trading_volume,
    columns=fb.index.month,
    colnames=['month'],
    normalize='columns'
)

month,1,2,3,4,5,6,7,8,9,10,11,12
trading_volume,,,,,,,,,,,,
low,0.95,1.00,0.71,0.95,1.00,1.00,0.86,1.00,1.00,1.00,1.00,1.00
med,0.05,0.00,0.19,0.05,0.00,0.00,0.10,0.00,0.00,0.00,0.00,0.00
high,0.00,0.00,0.10,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00


Si queremos realizar un cálculo que no sea contar la frecuencia, podemos pasar la columna sobre la que realizar el cálculo a `values` y la función a utilizar a `aggfunc`:

In [30]:
pd.crosstab(
    index=fb.trading_volume,
    columns=fb.index.month,
    colnames=['month'],
    values=fb.close,
    aggfunc=np.mean
)

C:\Users\feder\AppData\Local\Temp\ipykernel_127356\3586945560.py:1: FutureWarning: The provided callable <function mean at 0x000001874A8080E0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  pd.crosstab(


month,1,2,3,4,5,6,7,8,9,10,11,12
trading_volume,,,,,,,,,,,,
low,185.24,180.27,177.07,163.29,182.93,195.27,201.92,177.49,164.38,154.19,141.64,137.16
med,179.37,NaN,164.76,174.16,NaN,NaN,194.28,NaN,NaN,NaN,NaN,NaN
high,NaN,NaN,164.11,NaN,NaN,NaN,176.26,NaN,NaN,NaN,NaN,NaN


También podemos obtener subtotales de filas y columnas con el parámetro `margins`. Contemos el número de veces que cada estación registró nieve por mes e incluyamos los subtotales:

In [31]:
snow_data = weather.query('datatype == "SNOW"')
pd.crosstab(
    index=snow_data.station_name,
    columns=snow_data.index.month,
    colnames=['month'],
    values=snow_data.value,
    aggfunc=lambda x: (x > 0).sum(),
    margins=True, # show row and column subtotals
    margins_name='total observations of snow' # name the subtotals
)

month,1,2,3,4,5,6,7,8,9,10,11,12,total observations of snow
station_name,,,,,,,,,,,,,
"ALBERTSON 0.2 SSE, NY US",3.00,1.00,3.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,9
"AMITYVILLE 0.1 WSW, NY US",1.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3
"AMITYVILLE 0.6 NNE, NY US",3.00,1.00,3.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8
"ARMONK 0.3 SE, NY US",6.00,4.00,6.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,3.00,23
"BLOOMINGDALE 0.7 SSE, NJ US",2.00,1.00,3.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"WESTFIELD 0.6 NE, NJ US",3.00,0.00,4.00,1.00,0.00,NaN,0.00,0.00,0.00,NaN,1.00,NaN,9
"WOODBRIDGE TWP 1.1 ESE, NJ US",4.00,1.00,3.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,11
"WOODBRIDGE TWP 1.1 NNE, NJ US",2.00,1.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,7


<hr>
<div>
    <a href="./2-operaciones_con_dataframe.ipynb">
        <button>&#8592; Notebook Anterior</button>
    </a>
    <a href="./4-time_series.ipynb">
        <button style="float: right;">Proximo Notebook &#8594;</button>
    </a>
</div>
<hr>